# Nitsche's Method for boundary and interface conditions

In [ ]:
from netgen.occ import unit_square
from ngsolve import *
from ngsolve.webgui import Draw

mesh = Mesh(unit_square.GenerateMesh(maxh=0.05))

In [ ]:
fes = H1(mesh, order=4)
u,v = fes.TnT()

h = specialcf.mesh_size
a = BilinearForm(grad(u)*grad(v)*dx + 1/h*u*v*ds).Assemble()
f = LinearForm(10*v*dx).Assemble()

gfu = GridFunction(fes)
gfu.vec.data = a.mat.Inverse() * f.vec

print ("error bc:", Integrate((gfu-0)**2, mesh.Boundaries(".*")))
Draw (gfu);

In [ ]:
n = specialcf.normal(mesh.dim)

a = BilinearForm(fes)
a += grad(u)*grad(v)*dx + 1/h*u*v*ds
a += (-n*grad(u)*v-n*grad(v)*u) * ds(skeleton=True)
a.Assemble()

gfu.vec.data = a.mat.Inverse() * f.vec

print ("error bc:", Integrate((gfu-0)**2, mesh.Boundaries(".*")))
Draw (gfu);

## Interfaces